# Polarization Cell Model

This is a notebook trying to implement a [ Li | Electrolyte | Li ] model with full solute-volume-effects following Liu & Monroe's paper and the COMSOL model by Hou & Monroe. It will try follow the SEI growth example from model conception to solution within this notebook. 

## Dimensional Model Describing Binary Electrolyte Polarization

The model is a modified version of the concentrated-solution model originally implemented by Newman & Chapman. 

We want to make transient simulations of electrolyte behaviour by solving 5 coupled PDEs over a 1-D Cartesian geometry, with boundary conditions based on an applied current $I(t)$. This model assumes only lithium ions are involved in electrode reactions. 

We consider a 1-D Li-Li planar cell with electrolyte spanning from the lithium metal surface at $x=0$ through $x=L$ where $L$ is the thickness of the electrolyte.



#### 1. Cation Balance
The salt distribution follows this material balance:

nup*d(c,t)+d(Np,x)= 0

$$
\epsilon \frac{\partial c}{\partial t} = -\overrightarrow{\nabla} \cdot \frac{\overrightarrow{N}_+}{\nu_+}
$$

with a Dirichlet boundary condition:
$$
\overrightarrow{N}_+|_{x=0} = \frac{I(t)}{Fz_+A}
$$

#### 2. Charge Continuity
Charge is conserved, with current density $i = \frac{I(t)}{A}$, and $A$ being the cross-section area of the cell:
d(i,x) = 0


$$
\overrightarrow{\nabla}\cdot \overrightarrow{i} = 0
$$

with a Dirichlet boundary condition:
$$
\overrightarrow{i}|_{t={\textrm{pulse duration}}} = \frac{I(t)_\textrm{user defined}}{A}
$$

#### 3. Volume Continuity
Bulk flow of the solution:
Np*d(Vp,x)+Nm*d(Vm,x)+N0*d(V0,x)+d(vbox,x) = 0

$$
- \overrightarrow{\nabla} \cdot \overrightarrow{v}^\square = \frac{\bar{V}_{\textrm{e}} }{Fz_+ \nu_+} \overrightarrow{i} \cdot \overrightarrow{\nabla} t_+^0 + \frac{D_\textrm{eff} }{1 - \bar{V}_\textrm{e} c } \overrightarrow{\nabla} c \cdot \overrightarrow{\nabla} \bar{V}_\textrm{e}
$$

with a Dirichlet boundary condition:
$$
\left. \overrightarrow{v}^\square_x \right|_{x=0} = \frac{ \bar{V}_\textrm{e} \left( 1 - t_+^0 \right) I \left( t \right)}{F z_+ \nu_+ A},
$$

#### 4. MacInnes Equation
The local electrochemical state of the solution relates  to the current and composition gradients through a MacInnes equation: 

d(mup,x)/F/zp+i/kappa-(nup+num)*R*T*(1-tp0)*chi/cT/V0/F/zp/nup/c*d(c,x)


$$
- \overrightarrow{\nabla} \Phi = \frac{\overrightarrow{i}}{\kappa_\textrm{eff}} - \frac{\nu RT \chi ( 1 - t_+^0 )}{Fz_+ \nu_+ \left[ 1 + \left( \nu \bar{V}_0 - \bar{V}_\textrm{e} \right) c \right] } \overrightarrow{\nabla} \ln c
$$

voltage drop across the electrolyte can be computed by integrating from $0$ to $L$, initial value on a boundary is 0.

#### 5. Ficks Law
Cation flux can be driven by composition gradients, migration, or convection: 

Np+nup*D*chi*d(c,x)-tp0*i/F/zp-nup*c*vbox=0

$$
\overrightarrow{N}_+ = -\nu_+ D_{\textrm{eff}} \overrightarrow{\nabla} c + \frac{t_+^0}{Fz_+} \overrightarrow{i} + \nu_+ c \overrightarrow{v}^\square.
$$

Following this boundary condition 

Np+sp*i/F

## PyBaMM Model Implementation

Import pybamm and change working directory to root of the pybamm folder

In [75]:
import pybamm
import numpy as np
from scipy import constants
import matplotlib.pyplot as plt
import os
# os.chdir(pybamm.__path__[0]+'/..')
os.chdir('/Users/andrew/Dropbox/Research/DPhil/9 Pybamm_Modelling/PyBaMM')
# os.getcwd()

#### A model is defined in six steps:
1. Initialise model
2. Define parameters and variables
3. State governing equations
4. State boundary conditions
5. State initial conditions
6. State output variables

#### 1. Initialise model

In [83]:
model = pybamm.BaseModel()

#### 2. Define parameters and variables
Stating dimensional parameters, use `Parameter` object to create parameter symbols. Parameters which are functions are defined using  `FunctionParameter` object and should be defined within a python function as shown. 

In [77]:
# Defining Parameters

# PARAMETERS
#--------------------------------------------

# Galvanostatic Polarization (1. Pulse, 2. Relax)
t_pulse = pybamm.Parameter("Current pulse time [s]")
t_relax = pybamm.Parameter("Current off relaxation time [s]")
def I_applied(t_pulse,t_relax):
    "Current profile"
    inputs =  {"Current pulse time [s]": t_pulse,
               "Current off relaxation time [s]": t_relax,
              }
    return pybamm.FunctionParameter("Current profile [A]", inputs)

                                    
# Geometery
L = pybamm.Parameter("Electrolyte length [m]")
A = pybamm.Parameter("Electrode area [m2]")

# Physical constants
R = pybamm.Scalar(constants.R) #("Gas constant [J.mol-1.K-1]")
F = pybamm.Scalar(constants.physical_constants["Faraday constant"][0]) #("Faradays constant [C.mol-1]")
T = pybamm.Parameter("Temperature [K]")

# Stoich and charge constants
nu_p = pybamm.Parameter("Cation formula unit")
nu_m = pybamm.Parameter("Anion formula unit")
z_p = pybamm.Parameter("Cation equivalent charge")
z_m = pybamm.Parameter("Cation equivalent charge")
s_p = pybamm.Parameter("Cation stoichiometric coefficient")
s_n = pybamm.Parameter("Anion stoichiometric coefficient")
s_0 = pybamm.Parameter("Solvent stoichiometric coefficient")

# Electrolyte bulk property
c_avg = pybamm.Parameter("Bulk electrolyte concentration [mol.m-3]")
M_e = pybamm.Parameter("Salt molar mass [kg.mol-1]")
M_0 = pybamm.Parameter("Solvent molar mass [kg.mol-1]")

# Separator property
Br = pybamm.Parameter("Bruggeman coefficient")
eta  = pybamm.Parameter("Porosity")


# FUNCTION BASED PARAMETERS
#--------------------------------------------

# Composition based parameters - - -

def rho(c):
    "Density of electrolyte"
    inputs =  {"Li+ concentration [mol.m-3]": c,
              }
    return pybamm.FunctionParameter("Density [kg.m-3]", inputs)

def y(c):
    "Particle fraction"
    inputs =  {"Li+ concentration [mol.m-3]": c,
               "Solvent molar mass [kg.mol-1]": M_0,
               "Solute molar mass [kg.mol-1]": M_e,
              }
    return pybamm.FunctionParameter("Particle fraction", inputs)


# Thermodynamic properties - - - 

def V_e(c,rho,M_e):
    "Solute partial molar volume"
    inputs =  {"Li+ concentration [mol.m-3]": c,
               "Density [kg.m-3]": rho,
               "Salt molar mass [kg.mol-1]": M_e,
              }
    return pybamm.FunctionParameter("Solute partial molar volume [m3.mol-1]", inputs)


def V_0(c,rho,M_0):
    "Solvent partial molar volume"
    inputs =  {"Li+ concentration [mol.m-3]": c,
               "Density [kg.m-3]": rho,
               "Solvent molar mass [kg.mol-1]": M_0,
              }
    return pybamm.FunctionParameter("Solvent partial molar volume [m3.mol-1]", inputs)

def Chi(y):
    "Thermodynamic factor"
    inputs =  {"Particle fraction": y,
              }
    return pybamm.FunctionParameter("Thermodynamic factor", inputs)

# Transport properties - - - 

def EquivCond(y):
    "Equivalent conductivity"
    inputs =  {"Particle fraction": y,
              }
    return pybamm.FunctionParameter("Equivalent conductivity [S.m2.mol-1]", inputs)

def K_eff(EquivCond, Br, eta):
    "Effective conductivity"
    inputs =  {"Equivalent conductivity [S.m2.mol-1]": EquivCond,
               "Bruggeman coefficient": Br,
               "Porosity": eta,
              }
    return pybamm.FunctionParameter("Effective conductivity [S.m-1]", inputs)

def D_fick(y):
    "Fickian diffusivity"
    inputs =  {"Particle fraction": y,
              }
    return pybamm.FunctionParameter("Fickian diffusivity [m2.s-1]", inputs)

def D_eff(D_fick, Br, eta):
    "Effective diffusivity"
    inputs =  {"Fickian diffusivity [m2.s-1]": EquivCond,
               "Bruggeman coefficient": Br,
               "Porosity": eta,
              }
    return pybamm.FunctionParameter("Effective diffusivity [m2.s-1]", inputs)

def tp0(y):
    "Transference number"
    inputs =  {"Particle fraction": y,
              }
    return pybamm.FunctionParameter("Transference number", inputs)



In [78]:
# Defining variables

x = pybamm.SpatialVariable("x", domain="Electrolyte", coord_sys="cartesian")
c = pybamm.Variable("Salt concentration", domain="Electrolyte")

#### 3. State governing equations

Each governing equation must also be stated in the explicit form `d/dt = rhs` since pybamm only stores the right hand side (rhs) and assumes that the left hand side is the time derivative. The governing equations are then simply

In [81]:
# Algebraic relations?
# N_p = i_curr / (F * z_p)
i_curr = I_applied
nu = nu_p + nu_m

# 1. Cation Balance
dcdt = (-1/eta)*pybamm.div(N_p/nu_p)

# 2. Charge Continuity
div_i = pybamm.div(i_curr)

# 3. Volume Continuity
vol1 = pybamm.inner((V_e / (F * z_p * nu_p))*i_curr, pybamm.grad(tp0))
vol2 = pybamm.inner((D_eff / (1 - (V_e * c)))*pybamm.grad(c), pybamm.grad(V_e))
vol3 = pybamm.div(vbox)

vol_cont = vol1 + vol2 + vol3

# 4. MacInnes Equation
mac1 = i_curr/K_eff
mac2 = -((nu*R*T*Chi*(1-tp0))/(c*F*z_p*nu_p*(1+(nu*V_0-v_e)*c)))*pybamm.grad(c)
mac3 = pybamm.grad(Phi_ocv)

macinnes =  mac1 + mac2 + mac3

# 5. Ficks Law
fick1 = -nu_p*D_eff*pybamm.grad(c)
fick2 = (tp0/(F*z_p))*i_curr
fick3 = nu_p*c*vbox
fick4 = -N_p

fick = fick1 + fick2 + fick3 + fick4

# ADDING TO MODEL DICTIONARY
model.algebraic = {i_curr: div_i}
model.rhs = {N_p: dcdt}

NameError: name 'N_p' is not defined

{}